In [ ]:
import h5py
import meep as mp
import meep.adjoint as mpa
import numpy as np
from skimage.draw import disk
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import CenteredNorm, LogNorm, SymLogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import tikzplotlib

from util import h5_to_dict

In [ ]:
pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True,                # use LaTeX to write all text
    #"font.family": "lmodern",
    "font.serif": [],                   # blank entries should cause plots 
    "font.sans-serif": [],              # to inherit fonts from the document
    "font.monospace": [],
    "axes.labelsize": 10,               # LaTeX default is 10pt font.
    "font.size": 11,
    "legend.fontsize": 8,               # Make the legend/label fonts 
    "xtick.labelsize": 8,               # a little smaller
    "ytick.labelsize": 8,
    "pgf.preamble": " ".join([ # plots will use this preamble
        r"\usepackage[utf8]{inputenc}",
        r"\usepackage[T1]{fontenc}",
        #r"\usepackage{lmodern}",
        r"\usepackage{siunitx}",
        ])
    }
plt.rcParams.update(pgf_with_latex)
matplotlib.use("pgf")

In [ ]:
runs = h5_to_dict(h5py.File("index_delta_sweep_n1019_eval_res50.h5", "r"), {})
rr = runs[list(runs.keys())[0]]["radius"]
cf = len(runs[list(runs.keys())[0]]["ldos_freqs"]) // 2
cx, cy = np.array(runs[list(runs.keys())[0]]["fields_ex"].shape) // 2

## Simulation setup

In [ ]:
mp.verbosity(0)
sim_res = 50
radius = 5.7
pshape = (int(2 * sim_res * radius), int(2 * sim_res * radius))
lcen = 0.57
decay_by = 1e-6
index_low = 1.082
index_high = 1.167

d_wvg = 0.3
l_wvg = 1
dpml = 1.0
fcen = 1 / lcen
fwidth = 0.2

feature_size = 0.05

mask = np.zeros(pshape)
rr, cc = disk((shape - 1) / 2, shape[0] / 2, shape=shape)
mask[rr, cc] = 1

cavity_center = dpml + l_wvg + radius
cell = mp.Vector3(2 * (dpml + radius + 1), 2 * dpml + 2 * radius + 1 + l_wvg)

sources = [
    mp.Source(
        mp.GaussianSource(frequency=fcen, fwidth=fwidth),
        center=mp.Vector3(0, cavity_center),
        size=mp.Vector3(0, 0),
        component=mp.Ex,
    )
]

n1 = mp.Medium(index=index_low)
n2 = mp.Medium(index=index_high)

mg = mp.MaterialGrid(pshape, n1, n2)
mg.update_weights(np.ones_like(mask) * mask / 2)

design_region = mpa.DesignRegion(
    mg,
    volume=mp.Volume(
        center=mp.Vector3(0, cavity_center),
        size=mp.Vector3(2 * radius, 2 * radius),
    ),
)

geometry = [
    mp.Block(
        center=mp.Vector3(0, cavity_center),
        size=mp.Vector3(2 * radius, 2 * radius),
        material=mg,
    ),
    mp.Sphere(center=mp.Vector3(0, cavity_center), radius=0.4, material=n2),
    mp.Block(
        center=mp.Vector3(0, (dpml + l_wvg) / 2),
        size=mp.Vector3(d_wvg, dpml + l_wvg),
        material=n2,
    ),
]

sim = mp.Simulation(
    cell_size=cell,
    boundary_layers=[mp.PML(dpml)],
    geometry=geometry,
    sources=sources,
    resolution=sim_res,
    default_material=n1,
    geometry_center=mp.Vector3(0, cell.y / 2),
    symmetries=[mp.Mirror(mp.X, phase=-1)],
)

te0 = mpa.EigenmodeCoefficient(
    sim,
    mp.Volume(
        center=mp.Vector3(0, dpml),
        size=mp.Vector3(cell.x, 0),
    ),
    mode=1,
    eig_parity=mp.EVEN_Z,  # TM
)

mpa_opt = mpa.OptimizationProblem(
    simulation=sim,
    objective_functions=lambda x: anp.mean(anp.abs(x) ** 2),
    objective_arguments=[te0],
    design_regions=[design_region],
    frequencies=[fcen],
    decay_by=decay_by,
)

fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=300)
mpa_opt.plot2D(True, ax=ax)
fig.tight_layout()
fig.savefig("sim_setup.png", bbox_inches="tight", pad_inches=0, dpi=300)
plt.show()

## Cavities & Fields

In [ ]:
nx = ny = int(np.around(np.sqrt(len(runs))))
if len(runs) % ny != 0:
    ny += 1
    
extent = (-rr, rr, -rr, rr)

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(4, 2), dpi=300, sharex=True, sharey=True)
for axi, run in zip(np.array(ax).T, [runs[k] for k in ["0.01", "0.05", "0.09", "0.13"]]):
    axi[0].set_title(rf"$\Delta n = {np.around(run['index_delta'], 3)}$", fontsize=6, y=0.95)
    field = np.abs(run["fields_ex"])*2 + np.abs(run["fields_ey"])**2
    img = axi[1].imshow(field.T, origin="lower", cmap="magma", norm=LogNorm(1e-6, 1e6), extent=extent)
    design = run["design"]
    rc, cc = disk(np.array(design.shape) / 2, 21, shape=design.shape)
    design[rc, cc] = 1
    axi[0].imshow(design.T, origin="lower", cmap="gray_r", extent=extent)

fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.95, 0.24, 0.025, 0.6])
cbar_ax.tick_params(labelsize=5, width=0.6)
cbar_ax.set_title(r"$|E|^2$", fontsize=7, x=1.2)
fig.colorbar(img, cax=cbar_ax)

ax[0][0].set_ylabel(r"y (\unit{\um})", fontsize=6)
ax[1][0].set_ylabel(r"y (m)", fontsize=6)
ax[1][0].set_xlabel(r"x (m)", fontsize=6)
ax[1][1].set_xlabel(r"x (m)", fontsize=6)
ax[1][2].set_xlabel(r"x (m)", fontsize=6)
ax[1][3].set_xlabel(r"x (m)", fontsize=6)

#fig.suptitle("Designs & Fields", y=1.08,  fontsize=8)

for _, axi in np.ndenumerate(ax):
    axi.tick_params(axis="both", labelsize=5, width=0.5, length=2)
fig.subplots_adjust(hspace=0.0, wspace=0.2, top=0.95)
fig.savefig("fields_designs.png", bbox_inches="tight", pad_inches=0, dpi=300)
plt.show()

## Optimization history

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
color = plt.cm.turbo(np.linspace(1, 0, len(runs)))
for idx, (k, run) in enumerate(runs.items()):
    ax.plot(np.maximum.accumulate(run["hist"]), label=str(k), color=color[idx], alpha=1, zorder=100-idx)
ax.set_yscale("log")
ax.set_xlabel("n")
ax.set_ylabel("FoM")
plt.legend(title="Δn")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], title="Δn")
fig.tight_layout()
fig.savefig("optimization_hist.png", bbox_inches="tight", pad_inches=0, dpi=300)
plt.show()

## Purcell enhancement & coupling efficiency

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(6, 2.2), dpi=300)
color = plt.cm.turbo(np.linspace(1, 0, len(runs)))
for axi in ax:
    axi.tick_params(axis="both", labelsize=6)
    axi.set_xlabel(r"$\lambda$ (nm)", fontsize=7)
    axi.xaxis.set_ticks(np.linspace(540, 600, 5))
    axi.axvline(1000 / run["ldos_freqs"][cf], ls="--", color="gray", lw=1.0, alpha=0.5)
for idx, (k, run) in enumerate(runs.items()):
    ax[0].plot(
        1000 / run["ldos_freqs"],
        np.sum(np.abs(run["fluxes_design"]), axis=0) / np.sum(np.abs(run["fluxes_bulk"]), axis=0),
        color=color[idx],
        label=k,
        lw=0.7,
        alpha=1)
    ax[1].plot(
        1000 / run["ldos_freqs"],
        np.abs(run["mode_coeffs_design"][0, :, 1])**2 / np.sum(np.abs(run["fluxes_design"]), axis=0),
        color=color[idx],
        lw=0.7,
        alpha=1)
ax[0].set_ylabel("Purcell enhancement", fontsize=7)
ax[1].set_ylabel("Coupling efficiency", fontsize=7)
handles, labels = ax[0].get_legend_handles_labels()
ax[1].legend(
    handles[::-1], labels[::-1],
    title=r"$\Delta n$",
    title_fontsize=6,
    prop={"size": 5},
    ncol=1,
    fancybox=True,
    shadow=False,
    loc=0,
    bbox_to_anchor=(1.04, 1.04))
ax[0].set_title("(a)", loc="left", size=10)
ax[1].set_title("(b)", loc="left", size=10)
fig.tight_layout()
#fig.savefig("purcell_coupling_spectra.png", bbox_inches="tight", pad_inches=0, dpi=300)
plt.show()

In [ ]:
dn = [float(k) for k in runs.keys()]
purcell = [np.sum(np.abs(run["fluxes_design"]), axis=0)[cf] / np.sum(np.abs(run["fluxes_bulk"]), axis=0)[cf] for run in runs.values()]
coupling = [np.abs(run["mode_coeffs_design"][0, cf, 1])**2 / np.sum(np.abs(run["fluxes_design"]), axis=0)[cf] for run in runs.values()]
fig, ax1 = plt.subplots(1, 1, figsize=(3, 2), dpi=300)

color = "tab:blue"
ax1.plot(dn, purcell, ls="--", marker="x", lw=0.5, ms=3, color=color)
ax1.set_xlabel(r"$\Delta n$", fontsize=7)
ax1.set_ylabel("Purcell factor", color=color, fontsize=7)
ax1.tick_params(axis="x", labelsize=6)
ax1.tick_params(axis="y", labelcolor=color, labelsize=6)
ax1.xaxis.set_ticks(np.linspace(dn[0], dn[-1], 7))

color = "tab:red"
ax2 = ax1.twinx()
ax2.plot(dn, coupling, ls="--", marker="x", lw=0.5, ms=3, color=color)
ax2.set_ylabel("Coupling efficiency", color=color, fontsize=7)
ax2.tick_params(axis="y", labelcolor=color, labelsize=6)

fig.tight_layout()
fig.savefig("purcell_coupling_570nm.png", bbox_inches="tight", pad_inches=0, dpi=300)
plt.show()

## 3D

In [ ]:
data_3d = h5_to_dict(h5py.File("eval_3d_res80_n1019_nomode_dn0130.h5", "r"), {})
run = runs["0.13"]

In [ ]:
fields_xy = np.sum(np.abs(data_3d["fields_plane_xy_dr"]**2), axis=0)
fields_xz = np.sum(np.abs(data_3d["fields_plane_xz"]**2), axis=0)
fields_yz = np.sum(np.abs(data_3d["fields_plane_yz"]**2), axis=0)
freqs = data_3d["ldos_freqs"]
fluxes_bulk_3d = np.sum(np.abs(data_3d["fluxes_bulk"]), axis=0)
fluxes_design_3d = np.sum(np.abs(data_3d["fluxes_design"]), axis=0)
purcell_3d = fluxes_design_3d / fluxes_bulk_3d

purcell_2d = np.sum(np.abs(runs["0.13"]["fluxes_design"]), axis=0) / np.sum(np.abs(runs["0.13"]["fluxes_bulk"]), axis=0)
fields_2d = np.abs(runs["0.13"]["fields_ex"])**2 + np.abs(runs["0.13"]["fields_ey"])**2

fields_xy /= np.max(fields_xy)
fields_2d /= np.max(fields_2d)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(4, 2), dpi=300)
ax[0].imshow(fields_xy.T, origin="lower",  cmap="magma", norm=LogNorm(1e-7, 1))
ax[1].imshow(fields_2d.T, origin="lower",  cmap="magma", norm=LogNorm(3e-6, 1))
for axi in ax:
    axi.axis("off")
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
ax.imshow(np.sum(np.abs(data_3d["fields_plane_xz_wvg"]**2), axis=0).T[220:-100, 420:-420], origin="lower")
ax.axis("off")
plt.show()

In [ ]:
fluxes_bulk_3d_inplane = np.sum(np.abs(data_3d["fluxes_bulk"]), axis=0)
fluxes_design_3d_inplane = np.sum(np.abs(data_3d["fluxes_design"]), axis=0)
purcell_3d_inplane = fluxes_design_3d_inplane / fluxes_bulk_3d_inplane

In [ ]:
fig, ax = plt.subplots(6, 3, figsize=(12, 8))
for axi, bulk, design in zip(ax, data_3d["fluxes_bulk"], data_3d["fluxes_design"]):
    axi[0].plot(1000 / freqs, bulk)
    axi[1].plot(1000 / freqs, design)
    axi[2].plot(1000 / freqs, design / bulk)
fig.tight_layout()
plt.show()

In [ ]:
fluxes_bulk_inplane = np.sum(np.abs(np.take(data_3d["fluxes_bulk"], (0, 1, 3, 4), axis=0)), axis=0)
fluxes_design_inplane = np.sum(np.abs(np.take(data_3d["fluxes_design"], (0, 1, 3, 4), axis=0)), axis=0)
fluxes_bulk_oplane = np.sum(np.abs(np.take(data_3d["fluxes_bulk"], (2, 5), axis=0)), axis=0)
fluxes_design_oplane = np.sum(np.abs(np.take(data_3d["fluxes_design"], (2, 5), axis=0)), axis=0)
purcell_inplane = fluxes_design_inplane / fluxes_bulk_inplane
purcell_oplane = fluxes_design_oplane / fluxes_bulk_oplane

In [ ]:
plt.plot(1000 / freqs, np.roll(purcell_inplane, -8))
plt.plot(1000 / freqs, np.roll(purcell_oplane, -8))
plt.show()

In [ ]:
run = runs["0.13"]

fig, ax = plt.subplots(1, 2, figsize=(6, 2.2), dpi=300)
ax[0].plot(1000 / freqs, purcell_2d, lw=0.7)
ax[0].plot(1000 / freqs, np.roll(purcell_3d, -4), lw=0.7)
ax[0].axvline(570, color="gray", ls="--", lw=0.5)
ax[0].set_ylabel("Purcell factor", fontsize=7)

ax[1].plot(1000 / freqs, np.abs(run["mode_coeffs_design"][0, :, 1])**2 / np.sum(np.abs(run["fluxes_design"]), axis=0),
           label="2D",
           lw=0.7)
ax[1].plot(1000 / freqs, np.roll(np.abs(data_3d["wvg_flux"] / fluxes_design_3d), -4),
           label="3D",
           lw=0.7)
ax[1].set_ylabel("Coupling efficiency", fontsize=7)

for axi in ax:
    axi.xaxis.set_ticks(np.linspace(540, 600, 5))
    axi.axvline(570, color="gray", ls="--", lw=0.5)
    axi.tick_params(axis="both", labelsize=6)
    axi.set_xlabel(r"$\lambda$ (nm)", fontsize=7)

fig.legend(
    title_fontsize=6,
    prop={"size": 5},
    ncol=2,
    fancybox=True,
    shadow=False,
    loc=1,
    bbox_to_anchor=(0.96, 0.95))

ax[0].set_title("(a)", loc="left", size=10)
ax[1].set_title("(b)", loc="left", size=10)

fig.tight_layout()
fig.savefig("comparison_2d_3d.png", bbox_inches="tight", pad_inches=0, dpi=300)
plt.show()